# Create Heatmaps for E2F Transcription

This notebook looks at the significant genes in at least one cancer. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='/Enrichr')
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Regulation of RAC1 activity Homo sapiens 351aa...,20/38,1.112561e-08,0.000002,0,0,3.895750,71.346824,DOCK6;ARHGAP9;VAV3;RASGRF2;RAP1GDS1;VAV1;DEF6;...,NCI-Nature_2016
1,Class I PI3K signaling events Homo sapiens 12b...,19/48,6.480058e-06,0.000677,0,0,2.929928,35.003214,HSP90AA1;SYK;PLEKHA1;PDPK1;PTEN;PIK3CD;PIK3R1;...,NCI-Nature_2016
2,ATR signaling pathway Homo sapiens 8991cbac-61...,15/39,8.941964e-05,0.006230,0,0,2.846894,26.539231,RFC5;RFC3;TIPIN;RFC4;MCM7;RFC2;PLK1;RPA1;FANCD...,NCI-Nature_2016
3,E2F transcription factor network Homo sapiens ...,22/72,1.368548e-04,0.007151,0,0,2.261699,20.121410,PRMT5;RANBP1;RRM1;CDKN1B;CBX5;RRM2;APAF1;HDAC1...,NCI-Nature_2016
4,BCR signaling pathway Homo sapiens acbf44e2-61...,20/64,1.935528e-04,0.008091,0,0,2.313101,19.776925,MAP4K1;MAP2K1;SYK;CSNK2A1;CHUK;PDPK1;PTEN;PIK3...,NCI-Nature_2016


# Step 2: Get the list of significant genes from certain pathways

In [3]:
trans = prot_enr.res2d.Genes[3]
trans_genes = trans.split(';')
print('total genes:',len(trans_genes))

total genes: 22


# Step 3: Create HeatMap

Slice out the significant genes in the pathway from sig_df (sig_pval_heatmap.csv).

In [4]:
bool_df = sig_df.Proteomics.isin(trans_genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

22

In [5]:
# Only include p-values < 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= 0.01]

In [6]:
pu.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height=400, plot_width=600)

Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(trans_genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM3', 'RRM1', 'TOPBP1', 'TK1', 'HDAC1', 'YY1', 'E2F5', 'RRM2',
       'SP1'], dtype=object)

In [9]:
# Only include p-values < 0.01
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= 0.01]

In [10]:
pu.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=500)

Loading BokehJS ...

In [11]:
# all mcm and rfc 

In [12]:
df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')


In [13]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,21/134,4.645154e-13,1.361030e-10,0,0,7.626103,216.564409,DDX5;NCBP1;NCBP2;DDX23;DDX42;THOC3;PRPF40A;THO...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,12/36,3.876348e-12,5.678850e-10,0,0,16.220600,426.214559,RFC5;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;MCM3;MCM4;M...,KEGG_2016
2,RNA transport Homo sapiens hsa03013,19/172,2.855921e-09,2.789283e-07,0,0,5.375431,105.755548,RANBP2;EIF5B;NUP210;NUP155;NCBP1;NUP133;NCBP2;...,KEGG_2016
3,Ribosome biogenesis in eukaryotes Homo sapiens...,14/89,3.615167e-09,2.648110e-07,0,0,7.654665,148.792361,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,8/23,1.112006e-08,6.516354e-07,0,0,16.925844,309.988622,RFC5;MSH6;RFC3;PCNA;RFC4;RFC1;MSH2;RFC2,KEGG_2016


In [14]:
genes = prot_enr.res2d.Genes[1]
genes = genes.split(';')
genes

['RFC5',
 'RFC3',
 'PCNA',
 'RFC4',
 'MCM7',
 'RFC1',
 'RFC2',
 'MCM3',
 'MCM4',
 'MCM5',
 'MCM6',
 'MCM2']

In [15]:
get = df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_k = df[get] # Keep only genes with pos and neg
genes_k.Proteomics.unique()

array(['MCM6', 'MCM4', 'PCNA', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3',
       'RFC3', 'RFC2', 'RFC4', 'RFC1'], dtype=object)

In [16]:
# Only include p-values < 0.01
genes_k = genes_k.loc[genes_k['P_Value'] <= 0.01]

In [17]:
pu.plotCircleHeatMap(genes_k, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=800)

Loading BokehJS ...

In [18]:
# Look at p-values 
certain_gene = 'E2F5'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
1843,E2F5,0.007906,0.413613,Gbm
48605,E2F5,0.371901,-0.457100,Brca
60338,E2F5,0.641133,0.039176,Ov
66470,E2F5,0.078407,-0.282000,En
